# decompress petrel exported por and check with petrl

In [1]:
from CMG_format_decompress import CMG_format_decompress
import numpy as np
from pathlib import Path

In [70]:
file_path = Path('data/petrel_export_por/case38_por.GRDECL')
por_decompressed = CMG_format_decompress(file_path)
por_decompressed.shape

(989001,)

In [77]:
gridi = 41; gridj = 62; gridk = 70
p1=107*117*(gridk-1) + 107*(117-gridj) + gridi -1
print(por_decompressed[p1])


0.188128


In [60]:
#petrel grid (50,60,1), after export, grid (50,117-60,1)
file_path2 = Path('data/petrel_export_por/case38_JD_por.GRDECL')
por_decompressed2 = CMG_format_decompress(file_path2)
print(por_decompressed2.shape)
print(107*57+50)

(6149,)
6149


In [61]:
#petrel grid (20,40,1), after export, grid (50,117-40,1)
file_path3 = Path('data/petrel_export_por/case38_JD_por2.GRDECL')
por_decompressed3 = CMG_format_decompress(file_path3)
print(por_decompressed3.shape)
print(107*77+20)

(8259,)
8259


# check por extracted from FINIT

In [ ]:
por38 = np.load('results/properties/JD_BASECASE_38_PORO.npy')
por38_jreversed = np.load('results/properties/JD_BASECASE_38_jreversed_PORO.npy')
print(por38.shape)
print(por38_jreversed.shape)

(989001,)
(989001,)


In [78]:
gridi = 41; gridj = 62; gridk = 70
p1=107*117*(gridk-1) + 107*(117-gridj) + gridi -1
print(por38[p1])

0.21588497


In [79]:
gridi = 41; gridj = 62; gridk = 70
p1=107*117*(gridk-1) + 107*(gridj-1) + gridi -1
print(por38[p1])

0.22952856


In [80]:
gridi = 41; gridj = 62; gridk = 70
p1=107*117*(gridk-1) + 117*(gridi-1) + gridj -1
print(por38[p1])

0.29033941


In [81]:
gridi = 41; gridj = 62; gridk = 70
p1=107*117*(gridk-1) + 117*(107-gridi) + gridj -1
print(por38[p1])

0.31255639


# find a known grid cell por(20,40,2)=0.8888 in a Petrel exported por using cell origin (I=0, J=max, K)

Conclusion: when exporting properties from Petrel using cell origin (I=0, J=max, K), A cell (i, j, k) in Petrel corresponds to (i, j, k) in the exported file.

In [7]:
from CMG_format_decompress import CMG_format_decompress
import numpy as np
from pathlib import Path

file_path2 = Path('data/debug_jreverse/case38_JD_por3_jmax.GRDECL')
por_decompressed2 = CMG_format_decompress(file_path2)
print(por_decompressed2.shape)
idx = np.where(por_decompressed2==0.8888)
print(idx)

(989001,)
(array([16711]),)


In [8]:
print(idx[0]/(107*117))
print((idx[0]-(107*117))/107)
print((idx[0]-(107*117))%107)

[1.33485103]
[39.17757009]
[19]


In [9]:
gridi = 20; gridj = 40; gridk = 2
p1=107*117*(gridk-1) + 107*(gridj-1) + gridi -1
print(por_decompressed2[p1])

0.8888


# test reversing j colums uisng a simple array

Note: Petrel export traverse i first then j. Numpy flatten for 3d array follows rows

In [154]:
arr_shape=(2,3,4)
arr=np.arange(1,25)
arr

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [155]:
arr3d=arr.reshape(arr_shape[2],arr_shape[1],arr_shape[0])
arr3d = np.transpose(arr3d, axes=(0, 2, 1))
arr3d

array([[[ 1,  3,  5],
        [ 2,  4,  6]],

       [[ 7,  9, 11],
        [ 8, 10, 12]],

       [[13, 15, 17],
        [14, 16, 18]],

       [[19, 21, 23],
        [20, 22, 24]]])

In [156]:
arr3d = arr3d[:, :, ::-1]
arr3d

array([[[ 5,  3,  1],
        [ 6,  4,  2]],

       [[11,  9,  7],
        [12, 10,  8]],

       [[17, 15, 13],
        [18, 16, 14]],

       [[23, 21, 19],
        [24, 22, 20]]])

In [157]:
arr3d = np.transpose(arr3d, axes=(0, 2, 1))
arr3d

array([[[ 5,  6],
        [ 3,  4],
        [ 1,  2]],

       [[11, 12],
        [ 9, 10],
        [ 7,  8]],

       [[17, 18],
        [15, 16],
        [13, 14]],

       [[23, 24],
        [21, 22],
        [19, 20]]])

In [158]:
arr3d.flatten()

array([ 5,  6,  3,  4,  1,  2, 11, 12,  9, 10,  7,  8, 17, 18, 15, 16, 13,
       14, 23, 24, 21, 22, 19, 20])

# convert the Petrel exported por with a known grid cell por(20,40,2)=0.8888

Note: after reversing j columns, the location of a grid cell (i,j,k) (before reversing) after flattening into a 1d array is calculated as idx = 107*117*(k-1) + 107*(117-j) + i -1

In [ ]:
grid_shape=(107,117,79)
full_property = por_decompressed2.reshape(grid_shape[2], grid_shape[1], grid_shape[0])
full_property = np.transpose(full_property, axes=(0, 2, 1))
full_property = full_property[:, :, ::-1] # reverse j-direction
full_property = np.transpose(full_property, axes=(0, 2, 1))
full_property = full_property.flatten() # flatten back to 1D array
full_property.shape

(989001,)

In [11]:
gridi = 20; gridj = 40; gridk = 2
p2=107*117*(gridk-1) + 107*(117-gridj) + gridi -1
print(p2)
print(full_property[p2])

20777
0.8888


In [12]:
idx2=np.where(full_property==0.8888)
print(idx2)
print(idx2[0]/(107*117))
print((idx2[0]-(107*117))/107)
print((idx2[0]-(107*117))%107)

(array([20777]),)
[1.65963735]
[77.17757009]
[19]
